# 主題 04-1. 用RNN做情意分析

我們終於要介紹三大神經網路的最後一個, 也就是 RNN。RNN 有不少的變型, 例如 LSTM 和 GRU 等等, 不過我們都通稱叫 RNN。RNN 是一種「有記憶」的神經網路, 非常適合時間序列啦, 或是不定長度的輸入資料。

我們來看看怎麼樣用 RNN 做電影評論的「情意分析」, 也就是知道一則評論究竟是「正評」還是「負評」。

【註】因 TensorFlow 2 已做了一些改變, 例如完全整合了 Keras。到 2021 年的今天, 有一些細節也做了調整。因此我們依新的規範修改了程式。最大的不同是, 以後大家直接安裝 tensorflow 即可, 不用再另外裝 keras。

## 1. 初始準備

基本上和之前是一樣的, 我們就不再說明。

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

## 2. 讀入 IMDB 電影數據庫

今天我們要評入 IMDB 電影數據庫影評的部份。

In [ ]:
from tensorflow.keras.datasets import imdb

In [ ]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=10000)

In [ ]:
len(x_train)

25000

In [ ]:
len(x_test)

25000

### 2.1 輸入資料部份

我們來看一下輸入部份長什麼樣子?

In [ ]:
for i in range(10):
    print(len(x_train[i]), end=', ')

218, 189, 141, 550, 147, 43, 123, 562, 233, 130, 

### 2.2 輸出資料部份

輸出方面應該很容易想像, 我們來看看前 10 筆。結果自然就是 0 (負評) 或 1 (正評)。

In [ ]:
y_train[:10]

array([1, 0, 0, 1, 0, 0, 1, 0, 1, 0])

### 2.3 送入神經網路的輸入處理

雖然 RNN 是可以處理不同長度的輸入, 在寫程式時我們還是要

* 設輸入文字長度的上限
* 把每段文字都弄成一樣長, 太短的後面補上 0

In [ ]:
from tensorflow.keras.preprocessing import sequence

In [ ]:
x_train = sequence.pad_sequences(x_train, maxlen=100)
x_test = sequence.pad_sequences(x_test, maxlen=100)

In [ ]:
x_train.shape

(25000, 100)

至此我們可以來寫我們的第一個 RNN 了!

## 3. 打造你的 RNN

這裡我們選用 LSTM, 基本上用哪種 RNN 寫法都是差不多的!

### 3.1 決定神經網路架構

* 先將 10000 維的文字壓到 128 維 (one hot encoding)
* 然後用 128 個 LSTM (不一定要跟前面的128數字一樣)
* 最後一個 output, 直接用 sigmoid 送出 (用sigmoid因為最符合輸出是0-1的需求)

### 3.2 建構我們的神經網路

文字我們用 1-hot 表示是很標準的方式, 不過要注意的是, 因為我們指定要 1 萬個字, 所以每個字是用 1 萬維的向量表示! 這一來很浪費記憶空間, 二來字和字間基本上是沒有關係的。我們可以用某種「合理」的方式, 把字壓到比較小的維度, 這些向量又代表某些意思 (比如說兩個字代表的向量角度小表相關程度大) 等等。

這聽來很複雜的事叫 "word embedding", 而事實上 Keras 會幫我們做。我們只需告訴 Keras 原來最大的數字是多少 (10000), 還有我們打算壓到幾維 (128)。

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM

In [ ]:
model = Sequential()

In [ ]:
model.add(Embedding(10000, 128)) #把一個字從10000維壓到128維

In [ ]:
model.add(LSTM(150))

In [ ]:
model.add(Dense(1, activation='sigmoid'))

### 3.3 組裝

這次我們用 binary_crossentropy 做我們的 loss function, 另外用一個很潮的 Adam 學習法。

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 128)         1280000   
_________________________________________________________________
lstm (LSTM)                  (None, 150)               167400    
_________________________________________________________________
dense (Dense)                (None, 1)                 151       
Total params: 1,447,551
Trainable params: 1,447,551
Non-trainable params: 0
_________________________________________________________________


In [ ]:
3*(128+150+1)*150  #每個LSTM有三個gates(小小的神經元) 3*(128上層+150這層+1 bias)*150 

125550

In [ ]:
(128+150+1)*150 

41850

In [ ]:
125550 + 41850

167400

In [ ]:
model.compile(loss='binary_crossentropy',
            optimizer='adam',
            metrics=['accuracy'])

## 4. 訓練

In [ ]:
model.fit(x_train, y_train,
         batch_size=32,
         epochs=15)

Epoch 1/15
782/782 [==============================] - 10s 11ms/step - loss: 0.5123 - accuracy: 0.7313
Epoch 2/15
782/782 [==============================] - 8s 11ms/step - loss: 0.2607 - accuracy: 0.8948
Epoch 3/15
782/782 [==============================] - 8s 11ms/step - loss: 0.1821 - accuracy: 0.9326
Epoch 4/15
782/782 [==============================] - 8s 11ms/step - loss: 0.1138 - accuracy: 0.9597
Epoch 5/15
782/782 [==============================] - 8s 11ms/step - loss: 0.0812 - accuracy: 0.9734
Epoch 6/15
782/782 [==============================] - 8s 11ms/step - loss: 0.0611 - accuracy: 0.9797
Epoch 7/15
782/782 [==============================] - 8s 11ms/step - loss: 0.0385 - accuracy: 0.9887
Epoch 8/15
782/782 [==============================] - 8s 11ms/step - loss: 0.0260 - accuracy: 0.9918
Epoch 9/15
782/782 [==============================] - 8s 11ms/step - loss: 0.0260 - accuracy: 0.9914
Epoch 10/15
782/782 [==============================] - 8s 11ms/step - loss: 0.0190 - accur

## 5. 檢視結果

### 5.1 分數

我們照例來看看測試資料的分數。

In [ ]:
score = model.evaluate(x_test, y_test)

782/782 [==============================] - 3s 3ms/step - loss: 0.9819 - accuracy: 0.8173


In [ ]:
print('測試資料的 loss', score[0])
print('測試資料的正確率', score[1])

測試資料的 loss 0.9819172620773315
測試資料的正確率 0.8172799944877625


### 5.2 儲存結果

這裡有 8 成我們可以正確分辨, 看來還不差, 照例我們把結果存檔。

In [ ]:
model_json = model.to_json()
open('imdb_model_architecture.json', 'w').write(model_json)

1965

In [ ]:
model.save_weights('imdb_model_weights.h5') #把weight存起來

### 另一種存的方式

In [ ]:
model.save('myrnn.h5') #把weight跟架構全部都存起來了，下一次用load就可以取出來